### Traditional ML models test:

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from datasets import load_dataset
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

/workspaces/js-complexity-model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

import utils.evaluator as evaluator
evaluate = evaluator.evaluate

In [3]:
dataset = load_dataset("aslam-naseer/js-function-complexity-processed")
train = dataset["train"]
test = dataset["test"]
val = dataset["validation"]

In [4]:
df_train = train.to_pandas()
df_test = test.to_pandas()


X_train = df_train[:][['parameter_count', 'statement_count', 'variable_count', 'max_nesting_depth']]
y_train = df_train['complexity'] 
X_test = df_test[:][['parameter_count', 'statement_count', 'variable_count', 'max_nesting_depth']]
y_test = df_test['complexity']


In [5]:
def get_features(item):
    return  {
        'parameter_count': item['parameter_count'],
        'statement_count': item['statement_count'],
        'variable_count': item['variable_count'],
        'max_nesting_depth': item['max_nesting_depth']
    }

#### Linear Regression

In [6]:
np.random.seed(42)
features = ['parameter_count', 'statement_count', 'variable_count', 'max_nesting_depth']

model = LinearRegression()
model.fit(X_train, y_train)

for feature, coef in zip(features, model.coef_):
    print(f"{feature}: {coef:.3f}")
print(f"Intercept: {model.intercept_:.2f}")

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared Score: {r2:.2f}")

parameter_count: 0.065
statement_count: 0.101
variable_count: 0.038
max_nesting_depth: 0.116
Intercept: 2.50
Mean Squared Error: 1.86
R-squared Score: 0.59


In [7]:
def linear_regression(item):
    feature_vals = get_features(item)
    feature_df = pd.DataFrame([feature_vals])
    prediction = model.predict(feature_df)
    return float(prediction[0])

evaluate(linear_regression, test)

  0%|          | 0/500 [00:00<?, ?it/s]

0.6 1.4 0.3 1.0 0.7 1.7 0.1 1.0 0.0 0.8 0.3 0.6 2.0 0.3 0.2 5.0 5.3 0.2 2.3 0.5 0.5 1.5 0.1 0.3 0.0 1.9 1.7 0.2 0.3 0.2 0.2 2.3 2.6 0.2 0.3 1.3 0.0 0.3 3.3 0.4 1.6 2.4 0.6 1.9 0.9 0.8 0.3 0.5 0.4 2.0 0.4 0.4 0.9 0.3 0.1 1.9 0.9 1.3 1.1 0.1 0.7 0.2 0.1 0.1 1.7 0.2 0.4 0.3 0.0 0.1 0.5 0.3 1.8 3.9 1.0 0.7 0.9 0.7 0.3 1.7 0.1 0.2 1.1 0.3 0.4 0.4 0.9 0.5 0.6 1.0 1.4 1.0 1.1 1.2 0.9 2.5 0.1 0.7 0.8 2.2 0.2 0.1 0.2 0.9 0.3 5.0 2.6 2.2 0.0 0.7 0.6 0.3 1.2 0.5 0.7 0.2 1.6 1.8 1.6 0.9 1.6 0.4 0.5 1.8 0.2 0.8 0.4 0.1 0.3 0.4 0.5 0.2 0.2 0.7 2.0 0.0 1.2 0.4 1.0 0.1 0.7 0.7 0.7 0.7 0.5 0.1 2.5 0.0 0.5 0.2 0.3 2.2 0.7 1.1 0.4 0.7 2.7 0.5 0.2 0.2 2.6 0.2 0.5 1.0 0.2 1.0 0.5 0.1 0.6 0.2 0.2 1.0 1.4 0.1 0.8 1.8 2.1 0.7 0.9 0.3 0.6 0.2 0.2 1.7 0.8 0.7 1.8 0.1 0.2 0.0 1.9 0.2 0.1 0.7 3.4 2.3 1.2 0.0 0.7 0.0 0.7 0.4 1.3 1.7 0.2 0.2 0.6 1.9 0.3 3.1 1.8 0.4 0.5 0.9 0.3 0.6 0.3 0.5 0.3 2.0 0.3 0.2 

 45%|████▍     | 223/500 [00:00<00:00, 1028.79it/s]

0.7 

100%|██████████| 500/500 [00:00<00:00, 1667.98it/s]

2.4 1.2 0.8 0.3 2.9 5.1 1.4 0.3 0.7 1.1 1.8 0.7 0.3 0.2 0.3 1.3 0.3 0.3 0.6 2.2 1.1 1.0 0.1 0.3 0.7 0.3 0.7 0.2 0.1 0.4 1.4 0.7 0.4 0.0 1.6 1.1 0.5 0.9 0.3 0.8 0.1 0.2 0.9 0.2 0.1 0.2 0.6 0.6 2.8 2.2 1.7 0.7 0.0 0.0 0.1 0.3 0.7 0.5 1.4 0.1 2.8 1.4 0.1 0.8 1.4 2.3 2.0 0.2 0.3 3.1 0.4 0.9 2.4 1.4 2.1 0.6 1.4 0.3 1.6 0.3 2.4 0.3 0.5 0.6 0.6 1.4 1.2 0.2 0.6 0.8 0.8 0.1 1.3 0.5 0.9 0.6 1.5 0.4 0.4 0.6 0.8 0.1 1.6 0.3 2.5 0.1 1.8 0.0 0.4 2.2 0.9 0.1 0.7 0.0 0.8 0.4 0.8 1.8 1.1 0.1 0.8 1.1 0.6 0.0 3.7 1.6 0.1 0.6 2.0 3.7 3.8 1.4 1.2 1.0 2.3 1.2 0.9 0.2 0.3 0.7 0.3 3.1 0.1 1.0 0.5 0.2 0.2 1.7 1.3 0.9 1.5 1.5 1.6 0.3 0.1 0.6 2.1 4.2 0.3 1.9 0.8 0.3 0.6 3.0 0.2 0.3 0.2 2.4 0.3 1.5 3.2 0.4 0.2 2.3 1.7 1.3 0.2 0.0 2.7 1.1 3.7 0.3 0.4 3.3 0.2 3.7 0.1 1.1 0.1 0.0 0.1 1.7 0.7 0.6 2.2 4.2 3.2 0.3 0.7 0.2 0.2 1.1 1.2 0.7 1.2 1.7 3.5 1.1 0.3 1.4 1.6 0.9 0.5 1.1 0.6 0.8 0.8 5.3 0.5 0.4 0.6 0.4 1.2 1.3 0.3 0.6 1.0 0.2 0.7 0.2 0.2 0.7 0.3 0.4 0.4 1.1 1.9 0.4 0.3 0.5 3.2 0.3 0.2 0.3 0.8 3.7 2.2 2.2 1.9 0.7 

#### Random forest Ensamble

In [8]:
rf_model = RandomForestRegressor(n_estimators=20, random_state=42)
rf_model.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",20
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples

In [9]:
def rf_regression(item):
    feature_vals = get_features(item)
    feature_df = pd.DataFrame([feature_vals])
    prediction = rf_model.predict(feature_df)
    return float(prediction[0])

evaluate(rf_regression, test)

  7%|▋         | 34/500 [00:00<00:01, 246.53it/s]

0.2 1.7 0.5 1.3 0.2 0.9 1.0 1.2 0.6 1.6 0.5 1.0 3.2 0.1 0.6 5.6 4.9 0.6 2.6 0.1 0.2 0.5 0.5 0.6 0.4 1.3 2.2 2.4 0.6 0.2 0.6 2.5 2.0 0.1 1.3 1.6 0.6 0.4 2.4 0.4 0.0 3.4 1.4 2.2 2.1 0.3 0.4 0.1 0.6 3.0 0.4 0.3 1.1 1.1 0.3 1.8 0.8 1.0 0.2 0.6 0.1 0.0 0.3 0.1 1.0 0.0 0.6 0.5 0.9 

 23%|██▎       | 113/500 [00:00<00:01, 298.09it/s]

0.1 0.9 0.5 1.3 4.4 0.4 0.9 0.3 0.3 0.6 1.3 0.0 0.2 0.6 0.5 0.2 0.6 0.4 0.4 1.1 0.0 1.0 0.7 1.0 2.2 0.2 2.6 0.7 0.1 0.4 2.3 0.4 0.8 0.2 1.3 0.8 4.6 1.8 1.5 0.1 1.1 0.7 0.0 0.8 0.3 0.5 0.1 1.7 3.2 1.4 2.1 1.5 0.8 0.5 3.0 0.9 0.2 0.1 0.3 0.4 0.7 0.2 0.2 

 38%|███▊      | 192/500 [00:00<00:00, 323.62it/s]

...

0.5 0.6 1.7 1.2 0.2 0.4 0.7 1.1 1.0 2.4 0.4 0.0 0.4 1.2 1.2 1.0 0.6 1.7 0.9 0.7 1.1 0.8 0.6 0.0 0.8 4.4 0.7 1.3 0.0 1.7 0.0 2.6 0.8 0.0 0.5 2.5 0.1 1.2 0.9 0.2 0.1 3.0 1.4 0.4 1.5 0.9 1.9 0.9 3.7 0.3 0.4 2.8 0.2 3.6 0.3 1.8 1.0 0.0 0.2 1.0 1.2 3.1 1.2 4.5 3.6 1.0 0.0 0.5 

 93%|█████████▎| 466/500 [00:01<00:00, 334.97it/s]

0.4 0.2 1.2 0.3 1.3 2.1 3.8 1.2 0.7 1.4 1.9 0.7 0.7 0.9 0.8 1.0 0.6 2.1 0.2 0.1 1.2 0.6 2.1 1.8 2.2 0.2 1.2 0.0 0.3 1.0 0.4 0.2 0.7 0.2 0.2 0.7 1.7 0.4 0.9 1.0 1.0 0.3 0.0 0.3 0.8 3.6 1.9 1.8 1.9 0.7 0.3 0.1 1.1 1.7 0.0 0.5 2.1 3.6 1.4 0.1 0.0 0.8 0.9 0.7 

100%|██████████| 500/500 [00:01<00:00, 323.08it/s]


1.8 0.1 1.5 0.2 1.4 3.1 1.2 0.9 0.5 1.8 0.5 0.2 0.7 

#### XG Boost

In [26]:
np.random.seed(42)

xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=4, learning_rate=0.1)
xgb_model.fit(X_train, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes

In [27]:
def xgb_regression(item):
    feature_vals = get_features(item)
    feature_df = pd.DataFrame([feature_vals])
    prediction = xgb_model.predict(feature_df)
    return float(prediction[0])

evaluate(xgb_regression, test)

 16%|█▌        | 79/500 [00:00<00:00, 785.46it/s]

0.2 1.8 0.6 1.3 0.3 1.5 0.1 1.0 0.8 0.8 0.3 0.9 1.5 0.0 0.3 5.4 5.2 0.6 2.3 0.2 0.5 1.2 0.2 0.8 0.2 1.4 2.5 1.4 0.2 0.7 0.5 2.0 1.8 0.2 1.0 1.4 0.0 0.2 3.2 0.1 0.5 2.1 1.3 1.9 1.5 0.4 0.1 0.1 1.0 2.7 0.3 0.1 0.8 0.2 0.1 1.9 0.9 1.1 0.5 0.4 0.3 0.1 0.4 0.1 1.2 0.1 0.3 0.2 0.6 0.0 0.5 0.4 1.5 3.9 0.6 0.7 0.2 0.3 0.2 1.4 0.3 0.1 1.5 0.5 0.3 0.7 0.9 0.4 0.4 0.3 1.0 0.9 1.1 1.4 0.8 2.3 0.3 0.5 0.7 2.2 0.3 0.4 0.2 0.8 0.4 4.7 2.0 1.8 0.1 1.1 0.5 0.3 1.0 0.3 0.8 0.1 1.6 2.1 1.3 1.1 1.6 0.3 0.3 2.2 0.4 0.3 0.0 0.2 0.1 0.2 0.4 0.3 0.0 0.4 2.4 0.3 1.1 0.0 1.1 0.8 0.1 0.6 1.6 0.5 0.5 0.3 2.4 0.2 0.1 0.6 0.2 1.2 0.5 1.1 0.5 0.4 2.6 

 47%|████▋     | 235/500 [00:00<00:00, 718.25it/s]

1.2 0.4 0.2 2.7 0.1 0.5 1.2 0.6 1.0 0.3 0.1 0.4 0.4 0.4 1.6 1.4 0.4 0.5 2.0 1.3 0.6 0.0 0.4 0.5 0.3 1.1 2.1 0.4 0.1 1.7 0.3 0.8 0.1 1.7 0.3 0.2 0.7 3.3 2.4 0.5 0.2 0.0 0.2 0.6 1.3 0.9 1.7 0.4 0.2 0.4 2.1 0.9 3.6 1.4 0.4 0.3 0.5 0.6 0.8 1.0 0.1 0.7 1.7 0.0 0.6 0.5 0.9 0.8 1.1 0.0 2.6 4.8 1.2 0.1 0.3 0.7 2.1 0.4 0.2 1.0 0.3 1.9 0.2 0.9 0.7 1.9 1.0 0.7 0.7 0.1 0.5 0.7 0.7 1.9 0.0 0.3 0.7 0.4 0.7 0.1 1.6 1.0 1.0 1.1 0.8 0.8 0.2 0.2 0.3 0.1 0.1 0.1 0.4 0.5 2.4 2.9 2.5 0.7 0.5 0.4 1.0 0.1 0.3 0.7 1.4 0.0 2.4 1.5 0.0 0.7 0.9 1.4 0.8 0.0 0.3 2.9 0.6 0.6 1.3 1.1 1.0 

 76%|███████▌  | 381/500 [00:00<00:00, 700.82it/s]

0.4 1.1 0.2 1.6 0.0 3.0 0.3 0.1 1.0 0.3 0.8 1.3 0.3 1.1 0.6 0.5 0.1 0.3 1.0 1.0 0.8 1.0 0.7 0.4 0.4 1.1 0.2 1.9 0.1 1.9 0.1 2.1 0.4 0.0 2.6 0.7 0.1 0.8 0.2 0.5 0.6 0.5 1.1 0.7 0.1 0.5 1.0 0.3 0.5 3.8 1.6 0.2 1.3 1.5 3.0 3.6 1.7 1.5 1.2 1.9 0.9 0.9 0.2 0.3 0.1 0.5 3.0 0.2 0.2 0.7 0.5 0.6 1.8 0.9 1.3 1.0 1.1 1.6 0.2 0.2 0.2 1.4 4.6 0.2 1.4 0.2 0.1 0.3 2.7 0.1 0.1 0.5 1.7 0.0 1.3 0.7 0.2 0.2 2.8 1.5 1.6 0.3 0.2 2.2 0.4 3.6 0.7 0.1 3.1 0.2 3.9 0.2 0.8 0.1 0.2 0.3 1.2 1.0 0.6 1.6 4.2 3.3 0.3 1.2 0.0 0.3 0.8 0.7 0.6 0.8 2.1 3.5 0.4 0.7 1.3 2.0 0.7 0.6 0.8 0.5 0.7 0.4 3.0 

100%|██████████| 500/500 [00:00<00:00, 687.70it/s]


0.4 0.6 0.8 1.0 1.4 1.0 0.3 0.4 1.0 0.0 0.4 0.8 0.4 1.3 0.3 0.1 0.0 0.7 1.7 0.3 1.1 0.5 2.2 0.5 0.2 0.2 0.7 3.0 1.8 1.9 2.2 0.3 0.0 0.4 0.6 1.7 0.1 0.5 1.6 3.4 0.2 0.1 0.2 0.8 0.4 0.6 0.8 0.1 0.9 0.3 1.9 2.8 1.2 0.5 0.4 1.7 0.4 0.6 0.6 